## 0) Explore ETFdb (available filters + raw fields)
Use this to see what ETFdb actually returns in the public API.


In [ ]:
from etfs_analysis.etfdb import available_filters, fetch_top_by_category

filters = available_filters()
sorted(list(filters.keys()))


In [ ]:
# Example: show asset class values
filters.get('asset_class', {})


In [ ]:
# Sample records to see which fields are available
sample = fetch_top_by_category('asset_class', top_n=1)
sample.columns


In [ ]:
# Save a raw snapshot for manual inspection
sample.to_csv('etfdb_sample.csv', index=False)
sample.head(3)


# ETF Framework Examples
This notebook exercises the modular ETF analysis framework.


In [11]:
from etfs_analysis.config import Paths, Settings
from etfs_analysis.io import load_etf_returns, load_factors, load_etf_universe, save_etf_universe
from etfs_analysis.etfdb import build_universe
from etfs_analysis.prep import select_top_etfs_by_category, build_returns_panel
from etfs_analysis.simulation import simulate_portfolios
from etfs_analysis.optimization import annualize_stats, estimate_factor_model, factor_model_cov
from etfs_analysis.analysis import top_portfolio_overlap

paths = Paths()
settings = Settings()


## 1) Build or load ETF universe
Top-N ETFs per category from ETFdb.


In [12]:
if settings.refresh_universe or not paths.universe.exists():
    universe = build_universe(settings.category_fields, top_n=settings.top_n_per_category)
    save_etf_universe(universe, paths.universe)
else:
    universe = load_etf_universe(paths.universe)

universe.head()


,TICKER,NAME,AUM,ADV,ASSET_CLASS,CATEGORY_TYPE,CATEGORY,SOURCE
0,IBIT,iShares Bitcoin Trust ETF,6.728800e+10,64697879.0,Currency,asset_class,currency,ETFDB
1,FBTC,Fidelity Wise Origin Bitcoin Fund,1.760900e+10,6591486.0,Currency,asset_class,currency,ETFDB
2,GBTC,Grayscale Bitcoin Trust ETF,1.442800e+10,5641783.0,Currency,asset_class,currency,ETFDB
3,ETHA,iShares Ethereum Trust ETF,1.027700e+10,50309570.0,Currency,asset_class,currency,ETFDB
4,BTC,Grayscale Bitcoin Mini Trust ETF,4.232000e+09,2609962.0,Currency,asset_class,currency,ETFDB


## 2) Build return panel
Filter to top ETFs per category and create a date x ticker return matrix.


In [4]:
df_etf = load_etf_returns(paths.etf_returns)
top_etfs = select_top_etfs_by_category(universe, top_n=settings.top_n_per_category)
tickers = top_etfs["TICKER"].dropna().unique().tolist()
ret_panel = build_returns_panel(df_etf, tickers, min_history=settings.min_history, fill_method="none")
ret_panel


TICKER,AADR,AGG,AMLP,AOA,AOR,AVDE,AVDV,AVUV,BETZ,BIL,...,VTWO,VUG,VYM,XBI,XLG,XLK,XLRE,XLV,XMMO,ZIG
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,0.000400,0.000119,0.000324,0.000424,0.000342,0.000335,0.000404,0.000737,0.000325,0.000046,...,0.000492,0.000523,0.000394,0.000556,0.000486,0.000479,0.000354,0.000382,0.000678,0.000423
1993-02-01,0.000400,0.000119,0.000324,0.000424,0.000342,0.000335,0.000404,0.000737,0.000325,0.000046,...,0.000492,0.000523,0.000394,0.000556,0.000486,0.000479,0.000354,0.000382,0.000678,0.000423
1993-02-02,0.000400,0.000119,0.000324,0.000424,0.000342,0.000335,0.000404,0.000737,0.000325,0.000046,...,0.000492,0.000523,0.000394,0.000556,0.000486,0.000479,0.000354,0.000382,0.000678,0.000423
1993-02-03,0.000400,0.000119,0.000324,0.000424,0.000342,0.000335,0.000404,0.000737,0.000325,0.000046,...,0.000492,0.000523,0.000394,0.000556,0.000486,0.000479,0.000354,0.000382,0.000678,0.000423
1993-02-04,0.000400,0.000119,0.000324,0.000424,0.000342,0.000335,0.000404,0.000737,0.000325,0.000046,...,0.000492,0.000523,0.000394,0.000556,0.000486,0.000479,0.000354,0.000382,0.000678,0.000423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.007679,0.001138,0.015016,0.008687,0.005554,0.004248,0.005757,0.010303,0.002388,0.000219,...,0.009270,0.013482,0.009548,0.000979,0.012355,0.010333,0.006660,0.004108,0.008845,0.007569
2024-12-26,-0.000429,0.000723,-0.016026,-0.000129,0.000863,0.003254,0.004332,0.005562,0.006158,0.000219,...,0.010180,-0.001549,0.002791,0.007284,-0.000969,0.000665,0.001470,0.002010,0.003029,0.001401
2024-12-27,-0.006554,-0.002065,-0.000209,-0.006428,-0.005346,-0.000649,0.000154,-0.011576,-0.002873,0.000438,...,-0.014788,-0.014334,-0.005489,-0.014031,-0.012604,-0.013295,-0.008074,-0.004656,-0.010409,-0.010407


## 3) Portfolio simulations
Run equal-weight simulations across ETF counts.


In [5]:
factors = load_factors(paths.factors) if paths.factors.exists() else None
mkt_ret = None
if factors is not None and "mktrf" in factors.columns:
    mkt_ret = factors["mktrf"].astype(float) / 100.0

sim = simulate_portfolios(ret_panel, mkt_ret=mkt_ret, n_portfolios=settings.n_portfolios, etf_counts=settings.etf_counts)
sim


,n_etfs,tickers,ann_return,ann_vol,sharpe,max_dd,beta,mkt_var,idio_var,idio_share
0,5,"SSO,IWR,RSP,BND,IWM",0.116805,0.161034,0.725344,-0.571989,75.536776,7.775847e-05,0.000025,0.244359
1,5,"XLG,BNDX,MOTI,SVOL,SPXL",0.141301,0.104974,1.346051,-0.332575,37.815199,1.948784e-05,0.000024,0.554345
2,5,"MGK,UVXY,IWR,IJS,DGRO",-0.100375,0.128411,-0.781668,-0.973442,14.572295,2.893921e-06,0.000063,0.955773
3,5,"MTUM,EFAV,IYR,IWR,USMV",0.112887,0.112777,1.000971,-0.355317,48.512613,3.207304e-05,0.000018,0.364524
4,5,"DFAC,VBR,UVXY,GDX,REET",-0.129671,0.144491,-0.897433,-0.990005,-8.002607,8.727580e-07,0.000082,0.989465
...,...,...,...,...,...,...,...,...,...,...
895,20,"DIVO,CGBL,PFFA,IYR,BETZ,FTLS,HEFA,VYM,JEPQ,IWM...",0.155844,0.116259,1.340492,-0.358357,47.885356,3.124901e-05,0.000022,0.417380
896,20,"LRGF,GDX,SSO,QAI,VGT,GLDM,MOAT,XBI,TQQQ,PJAN,S...",0.133121,0.091574,1.453692,-0.255643,34.699544,1.640886e-05,0.000017,0.506903
897,20,"XLK,DXJ,AMLP,IEF,ITA,VOO,SLYV,JPST,HEFA,AOR,DE...",0.101731,0.092921,1.094806,-0.286563,41.307939,2.325401e-05,0.000011,0.321315
898,20,"AVDE,MOTO,HEDJ,QLD,BND,MOTI,VGIT,BNDX,SVOL,VTV...",0.090643,0.075790,1.195980,-0.283828,30.667006,1.281662e-05,0.000010,0.437717


## 4) Top risk-adjusted portfolios
Summaries of overlap and asset mix in the best Sharpe portfolios.


In [6]:
results = top_portfolio_overlap(sim, etf_universe=universe, top_pct=settings.top_pct)
results["top_tickers"]

,count,freq_top
IBIT,21,0.466667
CGBL,15,0.333333
CTA,8,0.177778
BUFR,8,0.177778
PFFA,8,0.177778
FAUG,7,0.155556
DBMF,7,0.155556
VCIT,7,0.155556
SCHD,7,0.155556
SPMO,7,0.155556


In [7]:
results["summary"]

,all,top
ann_return,0.101865,0.136767
ann_vol,0.093798,0.067466
sharpe,1.108802,2.050360
max_dd,-0.329676,-0.233924
idio_share,0.459074,0.568215


## 5) Factor model covariance (optional)
Compare sample covariance with factor-model covariance.


In [9]:
if factors is not None:
    factor_cols = [c for c in factors.columns if c.lower() not in ("rf",)]
    betas, fac_cov, idio_var = estimate_factor_model(ret_panel, factors, factor_cols=factor_cols)
    cov_f = factor_model_cov(betas, fac_cov, idio_var)
    cov_f
else:
    print('No factors available; skipped.')


In [10]:
cov_f

TICKER,AADR,AGG,AMLP,AOA,AOR,AVDE,AVDV,AVUV,BETZ,BIL,...,VTWO,VUG,VYM,XBI,XLG,XLK,XLRE,XLV,XMMO,ZIG
TICKER,,,,,,,,,,,,,,,,,,,,,
AADR,7.401502e-05,-4.469068e-07,1.592846e-05,1.987083e-05,1.460235e-05,8.693814e-06,8.506611e-06,1.366451e-05,8.226800e-06,-8.155890e-08,...,2.510710e-05,3.407811e-05,0.000027,3.782568e-05,3.003185e-05,4.815020e-05,1.115170e-05,3.084753e-05,1.279797e-05,9.750432e-06
AGG,-4.469068e-07,7.214644e-06,-9.641767e-07,-7.099792e-07,-5.053832e-07,-3.028932e-07,-3.483681e-07,-8.584976e-07,1.321648e-07,4.371394e-09,...,-7.824722e-07,-5.779834e-07,-0.000001,-2.814525e-07,-7.909788e-07,1.240390e-07,-2.677925e-07,-3.761478e-07,-3.520023e-07,-4.274941e-07
AMLP,1.592846e-05,-9.641767e-07,1.221020e-04,2.157376e-05,1.581932e-05,9.600674e-06,9.977169e-06,1.898294e-05,7.026892e-06,-1.037661e-07,...,2.687635e-05,3.105855e-05,0.000033,3.396831e-05,2.972508e-05,3.819523e-05,1.157028e-05,2.647902e-05,1.183949e-05,1.243252e-05
AOA,1.987083e-05,-7.099792e-07,2.157376e-05,4.851979e-05,1.793633e-05,1.077159e-05,1.081758e-05,1.859851e-05,9.038678e-06,-1.138647e-07,...,2.958917e-05,3.910879e-05,0.000036,4.131834e-05,3.612992e-05,5.334192e-05,1.365785e-05,3.492233e-05,1.409083e-05,1.305842e-05
AOR,1.460235e-05,-5.053832e-07,1.581932e-05,1.793633e-05,3.338091e-05,7.903203e-06,7.932003e-06,1.358257e-05,6.755811e-06,-8.490903e-08,...,2.171525e-05,2.883372e-05,0.000026,3.056864e-05,2.663408e-05,3.980883e-05,1.002299e-05,2.582971e-05,1.033172e-05,9.570777e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLK,4.815020e-05,1.240390e-07,3.819523e-05,5.334192e-05,3.980883e-05,2.231718e-05,2.121427e-05,2.832502e-05,2.717382e-05,-2.702056e-07,...,6.133076e-05,9.870691e-05,0.000073,1.077621e-04,8.723488e-05,2.161551e-04,3.089461e-05,9.626444e-05,3.110825e-05,2.373962e-05
XLRE,1.115170e-05,-2.677925e-07,1.157028e-05,1.365785e-05,1.002299e-05,6.230506e-06,6.174862e-06,1.026351e-05,5.240718e-06,-5.759511e-08,...,1.659491e-05,2.299746e-05,0.000020,2.332517e-05,2.091790e-05,3.089461e-05,5.002358e-05,2.040920e-05,8.022684e-06,7.574527e-06
XLV,3.084753e-05,-3.761478e-07,2.647902e-05,3.492233e-05,2.582971e-05,1.487639e-05,1.430593e-05,2.038789e-05,1.462062e-05,-1.707493e-07,...,3.929247e-05,6.260110e-05,0.000050,6.308204e-05,5.663815e-05,9.626444e-05,2.040920e-05,1.035188e-04,2.022720e-05,1.616629e-05


## 6) Factor correlations (factors only)


In [ ]:
from etfs_analysis.optimization import factor_correlation

if factors is not None:
    factor_correlation(factors)
else:
    print('No factors available; skipped.')


## 7) Fixed-ETF horizon simulations
Simulate 5-year outcomes for a chosen ETF list across random windows.


In [ ]:
from etfs_analysis.simulation import simulate_fixed_portfolio_horizons

# Example: pick 5 ETFs by ticker (edit this list)
selected = tickers[:5]
horizon_results = simulate_fixed_portfolio_horizons(ret_panel, selected, years=5, n_samples=100)
horizon_results.head()
